In [30]:
from mmseg.datasets import DynamicEarthNet
from mmengine.registry import init_default_scope
init_default_scope('mmseg')

data_root = '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly'
data_prefix=dict(img_path='planet', seg_map_path='labels')

In [31]:
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomCrop', crop_size=(512, 1024), cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

dataset = DynamicEarthNet(data_root=data_root, data_prefix=data_prefix,img_suffix='.tif', seg_map_suffix='.png',test_mode=False, pipeline=train_pipeline)

In [32]:
print(len(dataset))

1272


In [33]:
print(dataset.get_data_info(0))

{'img_path': '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/planet/10N_23E-188N_1311_3077_13_PF-SR_2018-01-01.tif', 'seg_map_path': '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/labels/10N_23E-188N_1311_3077_13_PF-SR_2018-01-01.png', 'label_map': None, 'reduce_zero_label': False, 'seg_fields': [], 'sample_idx': 0}


In [34]:
print(dataset.metainfo)

{'classes': ('impervious surface', 'agriculture', 'forest & other', 'wetland', 'soil', 'water'), 'palette': [[96, 96, 96], [204, 204, 0], [0, 204, 0], [0, 0, 153], [153, 76, 0], [0, 128, 255]], 'label_map': None, 'reduce_zero_label': False}


In [35]:
print(dataset[0])

{'inputs': tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8), 'data_samples': <SegDataSample(

    META INFORMATION
    seg_map_path: '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly/labels/10N_23E-188N_1311_3077_13_PF-SR_2018-01-01.png'
    ori_shape: (1024, 1024)
    reduce_zero_label: False
    img_shape: (512, 1024)
    f

In [4]:
from mmengine import Config
cfg = Config.fromfile('configs/pidnet/pidnet-s_2xb6-120k_1024x1024-cityscapes.py')
print(f'Config:\n{cfg.pretty_text}')

Config:
dataset_type = 'CityscapesDataset'
data_root = 'data/cityscapes/'
crop_size = (1024, 1024)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(
        type='RandomResize',
        scale=(2048, 1024),
        ratio_range=(0.5, 2.0),
        keep_ratio=True),
    dict(type='RandomCrop', crop_size=(1024, 1024), cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='GenerateEdge', edge_width=4),
    dict(type='PackSegInputs')
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(2048, 1024), keep_ratio=True),
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]
img_ratios = [0.5, 0.75, 1.0, 1.25, 1.5, 1.75]
tta_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(
        type='TestTimeAug',
        transforms=[[{
            'type': 'Resize',
            'scale_factor': 0.5,
            'keep_ratio': True


In [8]:
cfg.model.data_preprocessor.mean = [1042.59240722656, 915.618408203125, 671.260559082031, 2605.20922851562]
cfg.model.data_preprocessor.std = [957.958435058593, 715.548767089843, 596.943908691406, 1059.90319824218]
cfg.model.data_preprocessor.size=(1024, 1024)

cfg.model.backbone.in_channels=4
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomResize', scale=(1024, 1024), ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomCrop', crop_size=cfg.crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=(1024, 1024), keep_ratio=True),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]
cfg.work_dir = '.'
cfg.dataset_type = 'DynamicEarthNet'
cfg.data_root = '/home/Hung_Data/HungData/mmseg_data/Datasets/DynamicEarthNet/data_monthly'
cfg.train_dataloader.dataset.data_prefix=dict(img_path='planet', seg_map_path='labels')

In [9]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

05/21 11:40:49 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.8.16 (default, Mar  2 2023, 03:21:46) [GCC 11.2.0]
    CUDA available: True
    numpy_random_seed: 304
    GPU 0: NVIDIA TITAN RTX
    GPU 1: NVIDIA GeForce RTX 2080 Ti
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.1, V11.1.105
    GCC: gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0
    PyTorch: 1.10.1+cu102
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.2.3 (Git Hash 7336ca9f055cf1bfa13efb658fe15dc9b41f0740)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX512
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-

/home/Hung_Data/HungData/mmseg_data/mod_mmseg/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/Hung_Data/HungData/mmseg_data/mod_mmseg/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


05/21 11:42:08 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
05/21 11:42:12 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/home/Hung_Data/HungData/mmseg_data/mod_mmseg/mmseg/engine/hooks/visualization_hook.py:61: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '
